In [19]:
import torch
import torch.nn as nn
class simpleNet(nn.Module):
    def __init__(self,in_dim,n_hidden1,n_hidden2,out_dim):
        super(simpleNet,self).__init__()
        self.layer1=nn.Linear(in_dim,n_hidden1)
        self.layer2=nn.Linear(n_hidden1,n_hidden2)
        self.layer3=nn.Linear(n_hidden2,out_dim)
    def forward(self,x):
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        return x

In [20]:
class Activation_Net(nn.Module):
    def __init__(self,in_dim,n_hidden1,n_hidden2,out_dim):
        super(NeuralNetwork,self).__init__()
        self.layer1=nn.Sequential(nn.Linear(in_dim,n_hidden1),nn.ReLU(True))
        self.layer2=nn.Sequential(nn.Linear(n_hidden1,n_hidden2),nn.ReLU(True))
        self.layer3=nn.Sequential(nn.Linear(n_hidden2,out_dim))
    def forward(self,x):
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        return x

In [21]:
class Batch_Net(nn.Module):
    def __init__(self,in_dim,n_hidden1,n_hidden2,out_dim):
        super(Batch_Net,self).__init__()
        self.layer1=nn.Sequential(nn.Linear(in_dim,n_hidden1),nn.BatchNormld(n_hidden1),nn.ReLU(True))
        self.layer2=nn.Sequential(nn.Linear(n_hidden1,n_hidden2),nn.BatchNormld(n_hidden2),nn.ReLU(True))
        self.layer3=nn.Sequential(nn.Linear(n_hidden2,out_dim))
    def forward(self,x):
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        return x

In [22]:
#train
from torch import nn,optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets,transforms

batch_size=64
lr=1e-2
num_epoches=20

data_tf=transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5],[0.5])])

train_dataset=datasets.MNIST(root='./data',train=True,transform=data_tf,download=True)
test_dataset=datasets.MNIST(root='./data',train=False,transform=data_tf)
train_loade=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loade=DataLoader(test_dataset,batch_size=batch_size,shuffle=False)
model=simpleNet(28*28,300,100,10)
if torch.cuda.is_available():
    model=model.cuda()
    
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=lr)



In [25]:
# 训练模型
epoch = 0

# 选取所有训练数据集进行训练

for data in train_loade:
    # 获得img(手写图片)，label标签（手写图片对应数字）
    img, label = data
    img = img.view(img.size(0), -1)
    if torch.cuda.is_available():
        img = img.cuda()
        label = label.cuda()
    else:
        # 将数据转换成tensor类型
        img = Variable(img)
        label = Variable(label)
    
    #  向前传播，获得out结果和损失函数
    out = model(img)
    loss = criterion(out, label)
    print_loss = loss.data.item()
 
    # 反向传播，更新参数
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    epoch+=1
    if epoch%50 == 0:
        print('epoch: {}, loss: {:.4}'.format(epoch, loss.data.item()))


epoch: 50, loss: 1.991
epoch: 100, loss: 1.567
epoch: 150, loss: 1.158
epoch: 200, loss: 0.9498
epoch: 250, loss: 0.712
epoch: 300, loss: 0.6304
epoch: 350, loss: 0.6677
epoch: 400, loss: 0.8098
epoch: 450, loss: 0.62
epoch: 500, loss: 0.6052
epoch: 550, loss: 0.4616
epoch: 600, loss: 0.6785
epoch: 650, loss: 0.3786
epoch: 700, loss: 0.3607
epoch: 750, loss: 0.7172
epoch: 800, loss: 0.5133
epoch: 850, loss: 0.4969
epoch: 900, loss: 0.5016


In [28]:
model.eval()
eval_loss=0
eval_acc=0
for data in test_loade:
    img,label=data
    img=img.view(img.size(0),-1)
    if torch.cuda.is_available():
        img=Variable(img).cuda()
        label=Variable(label).cuda()
    else:
        img=Variable(img)
        label=Variable(label)
    out=model(img)
    loss=criterion(out,label)
    eval_loss+=loss.data.item()*label.size(0)#item是为了减少内存消耗
    _,pred=torch.max(out,1)
    num_correct=(pred==label).sum()
    eval_acc+=num_correct.item()
print('Test loss:{:.6f},Acc:{:.6f}'.format(
    eval_loss/(len(test_dataset)),
    eval_acc/(len(test_dataset))
    ))

Test loss:0.379145,Acc:0.893500
